In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import keras.utils.np_utils as kutils
from sklearn.model_selection import train_test_split
import keras.callbacks as callbacks
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.utils import plot_model
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
num_classes = 2

num_reps = 15

In [4]:
import random
random.seed(123)
import os

In [5]:
donut = np.load('donut.npy')
apple = np.load('apple.npy')
potato = np.load('potato.npy')
basketball = np.load('basketball.npy')
watermelon = np.load('watermelon.npy')
circle = np.load('circle.npy')
compass = np.load('compass.npy')
soccerball = np.load('soccer ball.npy')
wheel = np.load('wheel.npy')
cookie = np.load('cookie.npy')
classes = [apple, potato, basketball, watermelon, circle, compass, donut, soccerball, wheel, cookie]

In [6]:
batch_size = 128
epochs = 12

In [7]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(28,28,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(num_classes, activation='softmax'))

In [9]:
callbacks = [ModelCheckpoint('weights',
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                            mode='auto')
            ]
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["acc"])

In [10]:
test_eval = [None]*num_reps
used_classes = [None]*num_reps

for r in range(num_reps):
    random_classes = sorted(random.sample(range(0, 9), num_classes))
    used_classes[r] = random_classes
    for i,j in enumerate(random_classes):
        random_classes[i] = np.c_[classes[j], np.ones(classes[j].shape[0])*i]
    Data = np.concatenate(tuple(random_classes),axis=0)
    X = Data[:,0:784]
    Y = Data[:,784]
    del Data

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
    del X, Y
    X_train = X_train.reshape(-1, 28,28, 1)
    X_test = X_test.reshape(-1,28,28,1)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train = X_train / 255.
    X_test = X_test / 255.
    y_train = kutils.to_categorical(Y_train)
    y_test = kutils.to_categorical(Y_test)
    X_train,X_valid,y_train,y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13)

    model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs,
                       callbacks=callbacks,
                       validation_data=(X_valid, y_valid))

    test_eval[r] = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss (Loop ', r+1, '): ', test_eval[r][0], sep='')
    print('Test accuracy (Loop ', r+1, '): ', test_eval[r][1], sep='')

Train on 192670 samples, validate on 21408 samples
Epoch 1/12
192670/192670 [==============================] - 17s 89us/step - loss: 0.0501 - acc: 0.9807 - val_loss: 0.0409 - val_acc: 0.9833

Epoch 00001: val_acc improved from -inf to 0.98328, saving model to weights
Epoch 2/12
192670/192670 [==============================] - 16s 84us/step - loss: 0.0337 - acc: 0.9868 - val_loss: 0.0363 - val_acc: 0.9860

Epoch 00002: val_acc improved from 0.98328 to 0.98599, saving model to weights
Epoch 3/12
192670/192670 [==============================] - 16s 84us/step - loss: 0.0292 - acc: 0.9886 - val_loss: 0.0345 - val_acc: 0.9865

Epoch 00003: val_acc improved from 0.98599 to 0.98645, saving model to weights
Epoch 4/12
192670/192670 [==============================] - 16s 84us/step - loss: 0.0260 - acc: 0.9901 - val_loss: 0.0361 - val_acc: 0.9859

Epoch 00004: val_acc did not improve
Epoch 5/12
192670/192670 [==============================] - 16s 84us/step - loss: 0.0222 - acc: 0.9914 - val_loss:

205540/205540 [==============================] - 16s 80us/step - loss: 0.0039 - acc: 0.9985 - val_loss: 0.0270 - val_acc: 0.9929

Epoch 00006: val_acc did not improve
Epoch 7/12
205540/205540 [==============================] - 16s 80us/step - loss: 0.0043 - acc: 0.9985 - val_loss: 0.0291 - val_acc: 0.9935

Epoch 00007: val_acc improved from 0.99313 to 0.99352, saving model to weights
Epoch 8/12
205540/205540 [==============================] - 16s 80us/step - loss: 0.0037 - acc: 0.9986 - val_loss: 0.0372 - val_acc: 0.9926

Epoch 00008: val_acc did not improve
Epoch 9/12
205540/205540 [==============================] - 16s 80us/step - loss: 0.0024 - acc: 0.9991 - val_loss: 0.0382 - val_acc: 0.9923

Epoch 00009: val_acc did not improve
Epoch 10/12
205540/205540 [==============================] - 16s 80us/step - loss: 0.0032 - acc: 0.9989 - val_loss: 0.0394 - val_acc: 0.9921

Epoch 00010: val_acc did not improve
Epoch 11/12
205540/205540 [==============================] - 16s 80us/step - l

190272/190272 [==============================] - 16s 82us/step - loss: 0.1157 - acc: 0.9581 - val_loss: 0.0924 - val_acc: 0.9651

Epoch 00001: val_acc did not improve
Epoch 2/12
190272/190272 [==============================] - 16s 82us/step - loss: 0.0802 - acc: 0.9692 - val_loss: 0.0880 - val_acc: 0.9671

Epoch 00002: val_acc did not improve
Epoch 3/12
190272/190272 [==============================] - 16s 82us/step - loss: 0.0663 - acc: 0.9747 - val_loss: 0.0931 - val_acc: 0.9666

Epoch 00003: val_acc did not improve
Epoch 4/12
190272/190272 [==============================] - 16s 82us/step - loss: 0.0527 - acc: 0.9798 - val_loss: 0.0915 - val_acc: 0.9690

Epoch 00004: val_acc did not improve
Epoch 5/12
190272/190272 [==============================] - 15s 81us/step - loss: 0.0403 - acc: 0.9846 - val_loss: 0.1151 - val_acc: 0.9667

Epoch 00005: val_acc did not improve
Epoch 6/12
190272/190272 [==============================] - 16s 82us/step - loss: 0.0297 - acc: 0.9886 - val_loss: 0.1258


Epoch 00008: val_acc did not improve
Epoch 9/12
338367/338367 [==============================] - 28s 81us/step - loss: 0.0080 - acc: 0.9973 - val_loss: 0.0859 - val_acc: 0.9836

Epoch 00009: val_acc did not improve
Epoch 10/12
338367/338367 [==============================] - 28s 82us/step - loss: 0.0076 - acc: 0.9974 - val_loss: 0.0896 - val_acc: 0.9831

Epoch 00010: val_acc did not improve
Epoch 11/12
338367/338367 [==============================] - 28s 82us/step - loss: 0.0066 - acc: 0.9976 - val_loss: 0.1089 - val_acc: 0.9840

Epoch 00011: val_acc did not improve
Epoch 12/12
338367/338367 [==============================] - 28s 83us/step - loss: 0.0067 - acc: 0.9976 - val_loss: 0.1018 - val_acc: 0.9844

Epoch 00012: val_acc did not improve
Test loss (Loop 11): 0.1058001009156773
Test accuracy (Loop 11): 0.9837005670755711
Train on 341226 samples, validate on 37914 samples
Epoch 1/12
341226/341226 [==============================] - 29s 84us/step - loss: 0.0801 - acc: 0.9782 - val_los


Epoch 00003: val_acc did not improve
Epoch 4/12
333357/333357 [==============================] - 27s 82us/step - loss: 0.0328 - acc: 0.9885 - val_loss: 0.0479 - val_acc: 0.9840

Epoch 00004: val_acc did not improve
Epoch 5/12
333357/333357 [==============================] - 27s 80us/step - loss: 0.0280 - acc: 0.9903 - val_loss: 0.0496 - val_acc: 0.9849

Epoch 00005: val_acc did not improve
Epoch 6/12
333357/333357 [==============================] - 27s 80us/step - loss: 0.0232 - acc: 0.9920 - val_loss: 0.0608 - val_acc: 0.9830

Epoch 00006: val_acc did not improve
Epoch 7/12
333357/333357 [==============================] - 27s 80us/step - loss: 0.0189 - acc: 0.9936 - val_loss: 0.0662 - val_acc: 0.9842

Epoch 00007: val_acc did not improve
Epoch 8/12
333357/333357 [==============================] - 27s 80us/step - loss: 0.0157 - acc: 0.9947 - val_loss: 0.0655 - val_acc: 0.9841

Epoch 00008: val_acc did not improve
Epoch 9/12
333357/333357 [==============================] - 27s 80us/ste

In [11]:
Loss = np.array(test_eval)[:,0]
Accuracy = np.array(test_eval)[:,1]
print('Average Test loss:', np.mean(Loss))
print('Average Test accuracy:', np.mean(Accuracy))

Average Test loss: 0.15695058074093066
Average Test accuracy: 0.9677773576761471
